# Ricci Fragile Gas: 3D Visualization and Physics Application

This notebook provides interactive 3D visualizations of:
1. Walkers in flat Euclidean space
2. Walkers on the emergent Riemannian manifold
3. A real physics problem: Lennard-Jones cluster optimization

**Theory**: See `docs/source/12_fractal_gas.md`

**Implementation**: See `src/fragile/ricci_gas.py`

In [ ]:
import sys
import numpy as np
import torch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import holoviews as hv
from holoviews import opts
hv.extension('plotly')

# Add parent directory to path
sys.path.insert(0, '..')

from src.fragile.ricci_gas import (
    RicciGas,
    RicciGasParams,
    SwarmState,
    create_ricci_gas_variants,
    compute_kde_density,
    compute_kde_hessian,
    compute_ricci_proxy_3d,
)

print("✓ Imports successful")

## 1. Initialize Ricci Gas

In [ ]:
# Create Ricci Gas with moderate feedback strength
params = RicciGasParams(
    epsilon_R=0.5,           # Feedback strength (try varying: 0.1, 0.5, 1.0, 2.0)
    kde_bandwidth=0.4,       # Smoothing length
    epsilon_Ric=0.01,        # Regularization
    force_mode="pull",       # Gravity: toward high curvature
    reward_mode="inverse",   # Anti-gravity: reward low curvature
    R_crit=15.0,             # Singularity threshold
    gradient_clip=10.0,      # Numerical stability
)

gas = RicciGas(params)

print(f"Ricci Gas initialized:")
print(f"  Feedback strength α = {params.epsilon_R}")
print(f"  Smoothing length ℓ = {params.kde_bandwidth}")
print(f"  Force mode: {params.force_mode}")
print(f"  Reward mode: {params.reward_mode}")

## 2. Initialize Swarm

Start with walkers in a random configuration.

In [ ]:
N = 150  # Number of walkers
d = 3    # Dimension (always 3 for our implementation)

# Random initialization in [-2, 2]^3
torch.manual_seed(42)
x = torch.rand(N, d) * 4.0 - 2.0
v = torch.randn(N, d) * 0.1
s = torch.ones(N)

state = SwarmState(x=x, v=v, s=s)

print(f"Swarm initialized: {N} walkers in {d}D")
print(f"  Position range: [{x.min():.2f}, {x.max():.2f}]")
print(f"  Velocity std: {v.std():.3f}")

## 3. Compute Initial Geometry

In [ ]:
# Compute Ricci curvature and Hessian
R, H = gas.compute_curvature(state, cache=True)

print("Emergent geometry computed:")
print(f"  Ricci curvature R:")
print(f"    Min:  {R.min():.3f}")
print(f"    Mean: {R.mean():.3f}")
print(f"    Max:  {R.max():.3f}")
print(f"  Hessian eigenvalues (sample walker 0):")
eigenvals = torch.linalg.eigvalsh(H[0])
print(f"    λ = [{eigenvals[0]:.3f}, {eigenvals[1]:.3f}, {eigenvals[2]:.3f}]")

## 4. Visualization 1: Walkers in Flat Space

Visualize walkers in Euclidean 3D, colored by Ricci curvature.

In [ ]:
def plot_walkers_3d(state, title="Walkers in Flat Space"):
    """Plot walkers in 3D, colored by Ricci curvature."""
    x_np = state.x.detach().cpu().numpy()
    R_np = state.R.detach().cpu().numpy()
    alive = state.s.bool().cpu().numpy()
    
    # Alive walkers
    fig = go.Figure(data=[go.Scatter3d(
        x=x_np[alive, 0],
        y=x_np[alive, 1],
        z=x_np[alive, 2],
        mode='markers',
        marker=dict(
            size=5,
            color=R_np[alive],
            colorscale='RdBu_r',
            colorbar=dict(title="Ricci R"),
            line=dict(width=0.5, color='black'),
        ),
        text=[f"R={R_np[i]:.3f}" for i in np.where(alive)[0]],
        hovertemplate="<b>Walker %{text}</b><br>" +
                      "x: %{x:.2f}<br>" +
                      "y: %{y:.2f}<br>" +
                      "z: %{z:.2f}<extra></extra>",
        name="Alive",
    )])
    
    # Dead walkers (if any)
    if (~alive).any():
        fig.add_trace(go.Scatter3d(
            x=x_np[~alive, 0],
            y=x_np[~alive, 1],
            z=x_np[~alive, 2],
            mode='markers',
            marker=dict(size=3, color='gray', opacity=0.3),
            name="Dead",
        ))
    
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="x",
            yaxis_title="y",
            zaxis_title="z",
            aspectmode='cube',
        ),
        width=800,
        height=700,
    )
    
    return fig

fig = plot_walkers_3d(state, title="Initial Configuration: Walkers Colored by Ricci Curvature")
fig.show()

## 5. Run Dynamics

Evolve the swarm under Ricci-driven forces.

In [ ]:
# Run dynamics and track statistics
history = []
T = 300
dt = 0.1
gamma = 0.9  # Friction coefficient

print("Running dynamics...")
for t in range(T):
    # Compute geometry
    R, H = gas.compute_curvature(state, cache=True)
    
    # Compute force and reward
    force = gas.compute_force(state)
    reward = gas.compute_reward(state)
    
    # Simple Langevin dynamics (no cloning for simplicity)
    state.v = gamma * state.v + (1 - gamma) * force + torch.randn_like(state.v) * 0.05
    state.x = state.x + state.v * dt
    
    # Apply singularity regulation
    state = gas.apply_singularity_regulation(state)
    
    # Track statistics
    alive = state.s.bool()
    if alive.sum() > 0:
        variance = state.x[alive].var(dim=0).sum().item()
        R_mean = R[alive].mean().item()
        R_max = R[alive].max().item()
    else:
        variance = 0.0
        R_mean = 0.0
        R_max = 0.0
    
    history.append({
        't': t,
        'variance': variance,
        'R_mean': R_mean,
        'R_max': R_max,
        'alive_fraction': alive.float().mean().item(),
    })
    
    if t % 50 == 0:
        print(f"  t={t:3d}: var={variance:.3f}, R_mean={R_mean:.3f}, R_max={R_max:.3f}, alive={alive.sum()}/{N}")

print("\nDynamics complete!")

## 6. Visualization 2: Evolution Metrics

In [ ]:
# Extract time series
t_vals = [h['t'] for h in history]
variance = [h['variance'] for h in history]
R_mean = [h['R_mean'] for h in history]
R_max = [h['R_max'] for h in history]
alive_frac = [h['alive_fraction'] for h in history]

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Spatial Variance', 'Mean Ricci Curvature', 'Max Ricci Curvature', 'Alive Fraction'),
)

fig.add_trace(go.Scatter(x=t_vals, y=variance, mode='lines', name='Variance'), row=1, col=1)
fig.add_trace(go.Scatter(x=t_vals, y=R_mean, mode='lines', name='R_mean', line=dict(color='orange')), row=1, col=2)
fig.add_trace(go.Scatter(x=t_vals, y=R_max, mode='lines', name='R_max', line=dict(color='red')), row=2, col=1)
fig.add_trace(go.Scatter(x=t_vals, y=alive_frac, mode='lines', name='Alive', line=dict(color='green')), row=2, col=2)

fig.update_xaxes(title_text="Time", row=1, col=1)
fig.update_xaxes(title_text="Time", row=1, col=2)
fig.update_xaxes(title_text="Time", row=2, col=1)
fig.update_xaxes(title_text="Time", row=2, col=2)

fig.update_layout(height=700, showlegend=False, title_text="Evolution of Swarm Statistics")
fig.show()

# Interpretation
final_var = variance[-1]
if final_var < 0.5 * variance[0]:
    print("\n📉 SUPERCRITICAL REGIME: Variance collapsed (possible phase transition)")
elif final_var > 0.8 * variance[0]:
    print("\n🌊 SUBCRITICAL REGIME: Variance stable (diffuse gas phase)")
else:
    print("\n⚖️  NEAR-CRITICAL: Intermediate behavior")

## 7. Visualization 3: Final Configuration

In [ ]:
# Recompute final geometry
R_final, H_final = gas.compute_curvature(state, cache=True)

fig = plot_walkers_3d(state, title="Final Configuration: Emergent Structure")
fig.show()

print(f"\nFinal state:")
print(f"  Alive walkers: {state.s.sum():.0f}/{N}")
print(f"  Ricci range: [{R_final[state.s.bool()].min():.3f}, {R_final[state.s.bool()].max():.3f}]")
print(f"  Spatial std: {state.x[state.s.bool()].std(dim=0).mean():.3f}")

## 8. Visualization 4: Emergent Manifold

Visualize the emergent Riemannian metric via the **metric tensor eigenvalues**.

For each walker, the metric $g_i = H_i + \epsilon_\Sigma I$ defines local distances. We visualize:
- **Eigenvalue magnitudes** (size of ellipsoid axes)
- **Anisotropy** (ratio of max/min eigenvalue)

In [ ]:
def plot_emergent_manifold(state, epsilon_Sigma=0.01):
    """Visualize emergent metric via eigenvalue ellipsoids."""
    x_np = state.x.detach().cpu().numpy()
    H_np = state.H.detach().cpu().numpy()
    alive = state.s.bool().cpu().numpy()
    
    # Compute metric eigenvalues for alive walkers
    G = H_np + epsilon_Sigma * np.eye(3)  # g = H + ε I
    eigenvals = np.linalg.eigvalsh(G)  # [N, 3]
    
    # Anisotropy = max / min eigenvalue
    anisotropy = eigenvals[:, 2] / (eigenvals[:, 0] + 1e-8)
    
    fig = go.Figure()
    
    # Plot walkers, sized by mean eigenvalue, colored by anisotropy
    mean_eigval = eigenvals.mean(axis=1)
    
    fig.add_trace(go.Scatter3d(
        x=x_np[alive, 0],
        y=x_np[alive, 1],
        z=x_np[alive, 2],
        mode='markers',
        marker=dict(
            size=5 + 10 * (mean_eigval[alive] - mean_eigval[alive].min()) / (mean_eigval[alive].max() - mean_eigval[alive].min() + 1e-8),
            color=anisotropy[alive],
            colorscale='Viridis',
            colorbar=dict(title="Anisotropy<br>(λ_max/λ_min)"),
            line=dict(width=0.5, color='white'),
        ),
        text=[f"λ=[{eigenvals[i,0]:.2f}, {eigenvals[i,1]:.2f}, {eigenvals[i,2]:.2f}]" 
              for i in np.where(alive)[0]],
        hovertemplate="<b>%{text}</b><br>" +
                      "Anisotropy: %{marker.color:.2f}<extra></extra>",
        name="Metric Tensor",
    ))
    
    fig.update_layout(
        title="Emergent Riemannian Manifold: Metric Tensor Eigenvalues",
        scene=dict(
            xaxis_title="x",
            yaxis_title="y",
            zaxis_title="z",
            aspectmode='cube',
        ),
        width=800,
        height=700,
    )
    
    return fig

fig_manifold = plot_emergent_manifold(state, epsilon_Sigma=params.epsilon_Sigma)
fig_manifold.show()

print("\nInterpretation:")
print("  Marker size: Mean eigenvalue (local 'stiffness' of metric)")
print("  Marker color: Anisotropy (how elongated the metric ellipsoid is)")
print("  High anisotropy → Directional bias in geometry")

## 9. Visualization 5: Curvature Field

Create a 3D volume rendering of the Ricci curvature field.

In [ ]:
def create_curvature_isosurface(state, gas, grid_res=30, iso_level=None):
    """Create isosurface of Ricci curvature."""
    x_np = state.x.detach().cpu().numpy()
    alive = state.s.bool()
    
    # Define grid
    x_min, x_max = x_np[:, 0].min() - 1, x_np[:, 0].max() + 1
    y_min, y_max = x_np[:, 1].min() - 1, x_np[:, 1].max() + 1
    z_min, z_max = x_np[:, 2].min() - 1, x_np[:, 2].max() + 1
    
    x_grid = torch.linspace(x_min, x_max, grid_res)
    y_grid = torch.linspace(y_min, y_max, grid_res)
    z_grid = torch.linspace(z_min, z_max, grid_res)
    
    xx, yy, zz = torch.meshgrid(x_grid, y_grid, z_grid, indexing='ij')
    
    # Evaluation points
    x_eval = torch.stack([xx.flatten(), yy.flatten(), zz.flatten()], dim=-1)
    
    # Compute Hessian on grid (this is expensive!)
    print(f"Computing curvature on {len(x_eval)} grid points...")
    from src.fragile.ricci_gas import compute_kde_hessian, compute_ricci_proxy_3d
    
    H_grid = compute_kde_hessian(
        state.x,
        x_eval,
        gas.params.kde_bandwidth,
        alive,
    )
    R_grid = compute_ricci_proxy_3d(H_grid)
    R_grid = R_grid.reshape(grid_res, grid_res, grid_res)
    R_np = R_grid.detach().cpu().numpy()
    
    # Auto-select iso level if not provided
    if iso_level is None:
        iso_level = np.percentile(R_np, 75)  # 75th percentile
    
    print(f"Creating isosurface at R = {iso_level:.3f}")
    
    fig = go.Figure(data=go.Isosurface(
        x=xx.flatten().numpy(),
        y=yy.flatten().numpy(),
        z=zz.flatten().numpy(),
        value=R_np.flatten(),
        isomin=iso_level * 0.8,
        isomax=iso_level * 1.2,
        surface_count=3,
        colorscale='RdBu_r',
        colorbar=dict(title="Ricci R"),
        opacity=0.3,
        name="Curvature",
    ))
    
    # Add walkers
    fig.add_trace(go.Scatter3d(
        x=x_np[alive.cpu(), 0],
        y=x_np[alive.cpu(), 1],
        z=x_np[alive.cpu(), 2],
        mode='markers',
        marker=dict(size=3, color='black'),
        name="Walkers",
    ))
    
    fig.update_layout(
        title="Ricci Curvature Isosurface with Walkers",
        scene=dict(
            xaxis_title="x",
            yaxis_title="y",
            zaxis_title="z",
            aspectmode='cube',
        ),
        width=900,
        height=800,
    )
    
    return fig

# This is expensive - use low resolution for demo
print("⚠️  Warning: Isosurface computation is expensive. Using low resolution (20³ grid).")
print("   Increase grid_res for higher quality (but slower).\n")

fig_iso = create_curvature_isosurface(state, gas, grid_res=20)
fig_iso.show()

## 10. Real Physics Problem: Lennard-Jones Cluster Optimization

**Problem**: Find the minimum energy configuration of N particles interacting via Lennard-Jones potential.

**Lennard-Jones Potential**:
$$
V_{LJ}(r) = 4\epsilon \left[ \left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^{6} \right]
$$

**Total energy**:
$$
E = \sum_{i<j} V_{LJ}(\|x_i - x_j\|)
$$

**Challenge**: This has many local minima. Known global minima exist for small N (e.g., N=13 → icosahedron).

**Hypothesis**: The Ricci Gas can discover low-energy configurations by exploring negative curvature regions (saddle points connecting basins).

In [ ]:
def lennard_jones_energy(x, epsilon=1.0, sigma=1.0):
    """
    Compute Lennard-Jones energy for a set of particle positions.
    
    Args:
        x: [N, 3] particle positions
        epsilon: Energy scale
        sigma: Length scale
    
    Returns:
        E: Total energy (scalar)
        E_per_pair: [N, N] pairwise energies
    """
    N = len(x)
    
    # Pairwise distances [N, N]
    diff = x.unsqueeze(0) - x.unsqueeze(1)  # [N, N, 3]
    r = diff.norm(dim=-1)  # [N, N]
    
    # Avoid self-interaction
    r = r + torch.eye(N, device=x.device) * 1e10
    
    # Lennard-Jones potential
    r6 = (sigma / r) ** 6
    r12 = r6 ** 2
    
    V_pair = 4 * epsilon * (r12 - r6)
    
    # Total energy (sum over upper triangle to avoid double counting)
    mask = torch.triu(torch.ones(N, N, device=x.device), diagonal=1).bool()
    E = V_pair[mask].sum()
    
    return E, V_pair

def lennard_jones_force(x, epsilon=1.0, sigma=1.0):
    """
    Compute LJ force on each particle.
    
    Returns:
        F: [N, 3] forces
    """
    x = x.requires_grad_(True)
    E, _ = lennard_jones_energy(x, epsilon, sigma)
    
    F = -torch.autograd.grad(E, x)[0]
    
    return F

# Test
x_test = torch.randn(5, 3)
E_test, _ = lennard_jones_energy(x_test)
F_test = lennard_jones_force(x_test)

print(f"Lennard-Jones test:")
print(f"  5 particles: E = {E_test:.3f}")
print(f"  Force on particle 0: F = [{F_test[0,0]:.2f}, {F_test[0,1]:.2f}, {F_test[0,2]:.2f}]")

### Run Ricci Gas on Lennard-Jones Optimization

In [ ]:
# Initialize cluster
N_atoms = 13  # Classic LJ13 problem (known global min: icosahedron)

torch.manual_seed(123)
x_lj = torch.randn(N_atoms, 3) * 2.0  # Random initial configuration
v_lj = torch.zeros(N_atoms, 3)
s_lj = torch.ones(N_atoms)

state_lj = SwarmState(x=x_lj, v=v_lj, s=s_lj)

# Ricci Gas parameters for LJ optimization
params_lj = RicciGasParams(
    epsilon_R=0.3,           # Moderate curvature force
    kde_bandwidth=0.5,       # Smooth over ~2-3 particle spacings
    force_mode="pull",       # Aggregate toward high curvature
    reward_mode="inverse",   # Reward low curvature (exploration)
    R_crit=None,             # No singularity killing for LJ
)

gas_lj = RicciGas(params_lj)

print(f"Lennard-Jones Cluster Optimization: N = {N_atoms}")
print(f"  Initial energy: {lennard_jones_energy(x_lj)[0]:.3f}")
print(f"  Known global minimum (LJ13): E ≈ -44.327")
print(f"\nRunning Ricci Gas + LJ dynamics...\n")

In [ ]:
# Run optimization
history_lj = []
T_lj = 500
dt_lj = 0.05
gamma_lj = 0.8

best_E = float('inf')
best_x = None

for t in range(T_lj):
    # Compute Ricci geometry
    R_lj, H_lj = gas_lj.compute_curvature(state_lj, cache=True)
    F_ricci = gas_lj.compute_force(state_lj)
    
    # Compute LJ forces
    F_lj = lennard_jones_force(state_lj.x)
    
    # Combined dynamics: LJ force + Ricci curvature force
    F_total = F_lj + F_ricci
    
    # Langevin update
    state_lj.v = gamma_lj * state_lj.v + (1 - gamma_lj) * F_total + torch.randn_like(state_lj.v) * 0.1
    state_lj.x = state_lj.x + state_lj.v * dt_lj
    
    # Compute energy
    E_current, _ = lennard_jones_energy(state_lj.x)
    
    if E_current < best_E:
        best_E = E_current.item()
        best_x = state_lj.x.clone()
    
    # Track statistics
    history_lj.append({
        't': t,
        'E': E_current.item(),
        'E_best': best_E,
        'R_mean': R_lj.mean().item(),
        'R_max': R_lj.max().item(),
    })
    
    if t % 100 == 0:
        print(f"  t={t:3d}: E={E_current:.3f}, E_best={best_E:.3f}, R_mean={R_lj.mean():.2f}")

print(f"\nOptimization complete!")
print(f"  Best energy found: {best_E:.4f}")
print(f"  Known global min:  -44.327")
print(f"  Gap: {best_E - (-44.327):.4f}")

### Visualize LJ Optimization Results

In [ ]:
# Plot energy evolution
t_lj = [h['t'] for h in history_lj]
E_lj = [h['E'] for h in history_lj]
E_best_lj = [h['E_best'] for h in history_lj]

fig = go.Figure()
fig.add_trace(go.Scatter(x=t_lj, y=E_lj, mode='lines', name='Current E', line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=t_lj, y=E_best_lj, mode='lines', name='Best E', line=dict(color='red', width=2)))
fig.add_hline(y=-44.327, line_dash="dash", annotation_text="Global minimum", line_color="green")

fig.update_layout(
    title="Lennard-Jones Cluster Optimization (LJ13)",
    xaxis_title="Iteration",
    yaxis_title="Energy",
    width=900,
    height=500,
)
fig.show()

In [ ]:
# Visualize best configuration
x_best_np = best_x.detach().cpu().numpy()

# Compute pairwise distances
E_best, V_best = lennard_jones_energy(best_x)

fig = go.Figure()

# Draw atoms
fig.add_trace(go.Scatter3d(
    x=x_best_np[:, 0],
    y=x_best_np[:, 1],
    z=x_best_np[:, 2],
    mode='markers',
    marker=dict(size=15, color='blue', opacity=0.8, line=dict(width=2, color='darkblue')),
    name="Atoms",
))

# Draw bonds (for nearest neighbors, roughly r < 1.5σ)
diff = best_x.unsqueeze(0) - best_x.unsqueeze(1)
dist = diff.norm(dim=-1).cpu().numpy()

bond_threshold = 1.5  # Rough cutoff for visualization
for i in range(N_atoms):
    for j in range(i+1, N_atoms):
        if dist[i, j] < bond_threshold:
            fig.add_trace(go.Scatter3d(
                x=[x_best_np[i, 0], x_best_np[j, 0]],
                y=[x_best_np[i, 1], x_best_np[j, 1]],
                z=[x_best_np[i, 2], x_best_np[j, 2]],
                mode='lines',
                line=dict(color='gray', width=2),
                showlegend=False,
            ))

fig.update_layout(
    title=f"Best LJ13 Configuration Found (E = {best_E:.3f})",
    scene=dict(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="z",
        aspectmode='cube',
    ),
    width=800,
    height=700,
)
fig.show()

print(f"\nStructure analysis:")
print(f"  Center of mass: [{x_best_np.mean(axis=0)[0]:.3f}, {x_best_np.mean(axis=0)[1]:.3f}, {x_best_np.mean(axis=0)[2]:.3f}]")
print(f"  Radius of gyration: {np.sqrt(((x_best_np - x_best_np.mean(axis=0))**2).sum(axis=1).mean()):.3f}")
print(f"  Min pairwise distance: {dist[dist > 0].min():.3f}")
print(f"  Max pairwise distance: {dist.max():.3f}")

## 11. Summary and Next Steps

### What We've Demonstrated

1. **Flat Space Visualization**: Walkers colored by Ricci curvature
2. **Emergent Manifold**: Metric tensor eigenvalues showing geometric anisotropy
3. **Curvature Field**: 3D isosurfaces of Ricci scalar
4. **Phase Dynamics**: Evolution of variance, entropy, curvature
5. **Real Physics**: Lennard-Jones cluster optimization guided by curvature

### Key Observations

- **Phase behavior**: Depending on `epsilon_R` (α), the swarm either stays diffuse or collapses
- **Curvature guidance**: High curvature regions attract, low curvature regions disperse
- **LJ optimization**: The Ricci force helps escape local minima by exploring saddle points

### Experimental Directions

1. **Vary α**: Re-run with different `epsilon_R` values to find the phase transition
2. **Compare variants**: Test the 4 ablation study variants (Ricci, Aligned, Force-only, Reward-only)
3. **Larger LJ clusters**: Try N=19, 38, 55 (known difficult cases)
4. **Other physics problems**:
   - Protein folding (coarse-grained)
   - Rigid body packing
   - Molecular docking

### To Run More Experiments

```bash
# Full experimental suite
python experiments/ricci_gas_experiments.py --experiment all

# Or modify this notebook's parameters and re-run!
```

In [ ]:
print("\n" + "="*60)
print("  Ricci Fragile Gas: Visualization Complete")
print("="*60)
print(f"\n📊 Generated visualizations:")
print(f"  ✓ Walkers in flat space")
print(f"  ✓ Evolution metrics (variance, curvature, alive fraction)")
print(f"  ✓ Emergent Riemannian manifold (metric eigenvalues)")
print(f"  ✓ Curvature isosurface")
print(f"  ✓ Lennard-Jones cluster optimization")
print(f"\n🔬 Physics problem: LJ{N_atoms} cluster")
print(f"  Best energy: {best_E:.4f}")
print(f"  Gap to global: {best_E - (-44.327):.4f}")
print(f"\n📖 Theory: docs/source/12_fractal_gas.md")
print(f"💻 Code: src/fragile/ricci_gas.py")
print(f"\n🚀 Next: Try varying epsilon_R to explore phase transition!")
print("="*60)